<a href="https://colab.research.google.com/github/tinetor/MLOPS_FRAUD/blob/main/REST_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#TEST DATASET

import kagglehub

# Download latest version
path = kagglehub.dataset_download("whenamancodes/fraud-detection")

print("Path to dataset files:", path)


import pandas as pd
import os

# Assuming 'path' variable holds the correct directory path
try:
    # List files in the directory to verify
    print("Files in directory:", os.listdir(path))

    # Find the CSV file within the directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            csv_file_path = os.path.join(path, filename)
            break
    else:
        raise FileNotFoundError("No CSV file found in the specified directory.")

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    print("CSV file read successfully!")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

100%|██████████| 66.0M/66.0M [00:01<00:00, 55.4MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/whenamancodes/fraud-detection/versions/1
Files in directory: ['creditcard.csv']
CSV file read successfully!
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.37743

In [3]:
del df["Class"]


In [4]:
from pandas.io.json._table_schema import build_table_schema

In [5]:
schema=build_table_schema(df)

In [6]:
from pydantic import BaseModel, create_model

def create_pydantic_model_from_schema(schema: dict) -> BaseModel:
    # Mapeamento dos tipos do schema para tipos do Python
    type_mapping = {
        'integer': int,
        'number': float,
        'string': str,
        'boolean': bool
    }

    # Dicionário que armazenará os campos para o modelo Pydantic
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str se não encontrado
        # Como todos os campos são obrigatórios, usamos '...' para indicar isso
        fields[field_name] = (python_type, ...)

    # Cria e retorna o modelo Pydantic dinâmico
    DynamicModel = create_model('DynamicModel', **fields)
    return DynamicModel

In [7]:
DynamicModel = create_pydantic_model_from_schema(schema)

# Exemplo: visualizando o schema gerado
print(DynamicModel.schema_json(indent=2))

{
  "properties": {
    "index": {
      "title": "Index",
      "type": "integer"
    },
    "Time": {
      "title": "Time",
      "type": "number"
    },
    "V1": {
      "title": "V1",
      "type": "number"
    },
    "V2": {
      "title": "V2",
      "type": "number"
    },
    "V3": {
      "title": "V3",
      "type": "number"
    },
    "V4": {
      "title": "V4",
      "type": "number"
    },
    "V5": {
      "title": "V5",
      "type": "number"
    },
    "V6": {
      "title": "V6",
      "type": "number"
    },
    "V7": {
      "title": "V7",
      "type": "number"
    },
    "V8": {
      "title": "V8",
      "type": "number"
    },
    "V9": {
      "title": "V9",
      "type": "number"
    },
    "V10": {
      "title": "V10",
      "type": "number"
    },
    "V11": {
      "title": "V11",
      "type": "number"
    },
    "V12": {
      "title": "V12",
      "type": "number"
    },
    "V13": {
      "title": "V13",
      "type": "number"
    },
    "V14": {
   

<ipython-input-7-9bb93c494619>:4: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(DynamicModel.schema_json(indent=2))


In [8]:
!pip install dagshub mlflow
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.7/648.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import dagshub , mlflow
dagshub.init(repo_owner='tiago_netor', repo_name='antifraud_ml', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=fa78dd72-4335-4800-8749-cfaf682cc0b5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a2d4765ad8816f79191ae453250fff5c1c107eec24f2923985ea794feff3e510




Accessing as tiago_netor

Initialized MLflow to track repo "tiago_netor/antifraud_ml"

Repository tiago_netor/antifraud_ml initialized!

In [10]:
model_uri = "models:/MyXGBoostModel@staging"
model = mlflow.pyfunc.load_model(model_uri)




2025/02/26 00:06:19 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.2, required: mlflow==2.20.1)
 - xgboost (current: 2.1.4, required: xgboost==2.1.3)
 - scikit-learn (current: 1.6.1, required: scikit-learn==1.3.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [11]:
from fastapi import FastAPI
import uvicorn

In [12]:

# Mapeamento dos tipos de dados
type_mapping = {
    'integer': int,
    'number': float,
    'string': str,
    'boolean': bool
}

# Função para criar o modelo dinâmico
def create_pydantic_model_from_schema(schema: dict):
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str
        fields[field_name] = (python_type, ...)  # '...' torna o campo obrigatório
    return create_model('DynamicModel', **fields)

# Criação da aplicação FastAPI
app = FastAPI()

# Criando o modelo dinâmico
DynamicModel = create_pydantic_model_from_schema(schema)
@app.post("/predict")
async def predict(item: DynamicModel):
    # Preparando os dados para previsão
    input_data = np.array([[item.Time, item.V1, item.V2]])  # Ajuste conforme o seu schema
    prediction = model.predict(input_data)
    return {"prediction": prediction.tolist()}



In [13]:
# EXTRA

In [14]:
!pip install pyngrok
from pyngrok import ngrok

In [15]:
import threading

In [16]:
import os
import sys
import asyncio
import threading
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pyngrok import ngrok

# Mapeamento dos tipos de dados
type_mapping = {
    'integer': int,
    'number': float,
    'string': str,
    'boolean': bool
}

# Função para criar o modelo dinâmico
def create_pydantic_model_from_schema(schema: dict):
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str
        fields[field_name] = (python_type, ...)  # '...' torna o campo obrigatório
    return create_model('DynamicModel', **fields)

# Criação da aplicação FastAPI
app = FastAPI()

# Criando o modelo dinâmico
DynamicModel = create_pydantic_model_from_schema(schema)

@app.get("/")
async def home():
    return {"message": "API funcionando!"}
@app.post("/predict")
async def predict(item: DynamicModel):
    # Preparando os dados para previsão
    input_data = np.array([[item.Time, item.V1, item.V2]])  # Ajuste conforme o seu schema
    prediction = model.predict(input_data)
    return {"prediction": prediction.tolist()}


# Permitir que o Uvicorn rode no Colab sem conflitos de loop
nest_asyncio.apply()

# Função para rodar o servidor Uvicorn
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Rodar o servidor Uvicorn em uma thread separada
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Autenticação do ngrok
ngrok.set_auth_token("")  # Substitua pela sua chave
ngrok.kill()
# Criar túnel com ngrok
public_url = ngrok.connect(8000).public_url
print(f"🚀 API rodando em: {public_url}")


INFO:     Started server process [253]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 API rodando em: https://581d-34-48-14-135.ngrok-free.app
